In [2]:
import gym
import numpy as np

#
# Environment
#
env = gym.make('CartPole-v1')
state = env.reset()
action = env.action_space.sample()

print('State space: ', env.observation_space)
print('Initial state: ', state)
print('\nAction space: ', env.action_space)
print('Random action: ', action)

State space:  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Initial state:  [-0.03554296  0.00394534  0.03405297  0.02208636]

Action space:  Discrete(2)
Random action:  1


In [3]:
# DQN Modeling
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

num_state = env.observation_space.shape[0]
num_action = env.action_space.n

model = Sequential()
model.add(Dense(24, input_dim= num_state, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(num_action, activation='linear'))
model.compile(loss='mse', optimizer="adam")


/Users/mash/Projects/mash-ML/venv-mac/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/Users/mash/Projects/mash-ML/venv-mac/lib/python3.9/site-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/mash/Projects/mash-ML/venv-mac/lib/python3.9/site-packages/keras/utils/image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/mash/Projects/mash-ML/venv-mac/lib/python3.9/site-packages/keras/utils/image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/

In [5]:
import random
from collections import deque
from tqdm import tqdm

num_episode = 10
memory = deque(maxlen=2000)

# Hyper parameter
epsilon = 0.3
gamma = 0.95
batch_size = 32

# DQN Learning
for episode in tqdm(range(num_episode)):
    state = env.reset()
    done = False
    while not done:
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            q_value = model.predict(state.reshape(1, num_state))
            action = np.argmax(q_value[0])
        next_state, reward, done, info = env.step(action)
        # Memory
        memory.append((state, action, reward, next_state, done))
        
        state = next_state
    
    # Replay
    if len(memory) > batch_size:
        mini_batch = random.sample(memory, batch_size)
        for state, action, reward, next_state, done in mini_batch:
            if done:
                target = reward
            else:
                target = reward + gamma * (np.max(model.predict(next_state.reshape(1, num_state))[0]))
            q_value = model.predict(state.reshape(1, num_state))
            q_value[0][action] = target
            model.fit(state.reshape(1, num_state), q_value, epochs=1, verbose=0)

env.close()

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 15ms/step


 10%|█         | 1/10 [00:00<00:03,  2.25it/s]

1/1 [==============================] - 0s 15ms/step


 20%|██        | 2/10 [00:00<00:02,  2.89it/s]

...

 80%|████████  | 8/10 [00:25<00:07,  3.93s/it]

1/1 [==============================] - 0s 15ms/step


 90%|█████████ | 9/10 [00:30<00:04,  4.08s/it]

1/1 [==============================] - 0s 14ms/step


100%|██████████| 10/10 [00:34<00:00,  3.41s/it]


In [6]:
import os

save_dir = os.getcwd()
model_name = 'keras_dqn_trained_model.h5'

# Save model and weights
model_path = os.path.join(save_dir, model_name)
model.save(model_path)


# https://github.com/skettee/notebooks/blob/master/deep_q_network.ipynb